In [ ]:
# #colabを使う方はこちらを使用ください。
# !pip install torch==0.4.1
# !pip install torchvision==0.2.1
# !pip install numpy==1.14.6
# !pip install matplotlib==2.1.2
# !pip install pillow==5.0.0
# !pip install opencv-python==3.4.3.18
# !pip install torchtext==0.3.1

In [4]:
import torch
import torch.nn as nn

# 多対多のリカレントニューラルネットワーク

## シーケンス長が揃っていないデータにtorch.nn.LSTＭを使用

In [5]:
seq_len_list = [8, 6, 10]

input_size = 2
hidden_size = 4

# 入力データのサンプル
input_seq = []
for seq_len in seq_len_list:
    iseq = torch.randn(seq_len, input_size)
    print('input sequence:', iseq.shape)
    input_seq.append(iseq)

# ネットワークの定義
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝播
output_seq = []     # 出力データを格納するリスト
for iseq in input_seq:
    iseq = iseq.unsqueeze(0)    # 先頭にバッチサイズの次元1を加える
    oseq, _ = net(iseq)
    output_seq.append(oseq.squeeze(0)) # squeezeで先頭のバッチサイズの次元1を削除してリストに追加

# 出力データの確認
for oseq in output_seq:
    print('output sequence:', oseq.shape)

input sequence: torch.Size([8, 2])
input sequence: torch.Size([6, 2])
input sequence: torch.Size([10, 2])
output sequence: torch.Size([8, 4])
output sequence: torch.Size([6, 4])
output sequence: torch.Size([10, 4])


## シーケンス長が揃っているデータにtorch.nn.LSTＭを使用

In [6]:
batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# 入力データのサンプル
input_seq = torch.randn(batch_size, seq_len, input_size)
print('input sequence:', input_seq.shape)

# ネットワークの定義
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝播
output_seq, _ = net(input_seq)

# 出力データの確認
print('output sequence:', output_seq.shape)

input sequence: torch.Size([20, 10, 2])
output sequence: torch.Size([20, 10, 4])


## シーケンス長が揃っているデータにtorch.nn.LSTMCellを使用

In [7]:
batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# 入力データのサンプル
input_seq = torch.randn(batch_size, seq_len, input_size)
print("input sequence：", input_seq.size())


# ネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block_a = nn.LSTMCell(input_size, hidden_size)     # LSTMブロックA
        self.block_b = nn.LSTMCell(hidden_size, hidden_size)    # LSTMブロックB
    
    def forward(self, x, hx_a0, cx_a0, hx_b0, cx_b0):
        '''
        :param x: 現時刻の入力シーケンスの状態
        :param hx_a0: 前時刻のLSTMブロックAの隠れ層の状態
        :param cx_a0: 前時刻のLSTMブロックAのセル状態
        :param hx_b0: 前時刻のLSTMブロックBの隠れ層の状態
        :param cx_b0: 前時刻のLSTMブロックBのセル状態
        :return: 現時刻のLSTMブロックAの隠れ層の状態とセル状態、現時刻のLSTMブロックBの隠れ層の状態とセル状態
        '''
        hx_a1, cx_a1 = self.block_a(x, (hx_a0, cx_a0))      # 現時刻のLSTMブロックAの隠れ層の状態とセル状態
        hx_b1, cx_b1 = self.block_b(hx_a1, (hx_b0, cx_b0))  # 現時刻のLSTMブロックBの隠れ層の状態とセル状態
        return hx_a1, cx_a1, hx_b1, cx_b1


net = Net()

# 隠れ層の初期化
hx_a = torch.randn(batch_size, hidden_size)
cx_a = torch.randn(batch_size, hidden_size)
hx_b = torch.randn(batch_size, hidden_size)
cx_b = torch.randn(batch_size, hidden_size)

# 順伝播
output_seq = []     # 出力データを格納するリスト
for i in range(seq_len):
    hx_a, cx_a, hx_b, cx_b = net(input_seq[:, i, :], hx_a, cx_a, hx_b, cx_b)
    output_seq.append(hx_b)   # リストに追加
output_seq = torch.stack(output_seq, dim=1)     # リストからTensorに変換

# 出力データの確認
print('output sequence:', output_seq.shape)

input sequence： torch.Size([20, 10, 2])
output sequence: torch.Size([20, 10, 4])


# 多対一のリカレントニューラルネットワーク

## シーケンス長が揃っていないデータにtorch.nn.LSTＭを使用

In [8]:
seq_len_list = [8, 6, 10]

input_size = 2
hidden_size = 4

# 入力データのサンプル
input_seq = []
for seq_len in seq_len_list:
    iseq = torch.randn(seq_len, input_size)
    print('input sequence:', iseq.shape)
    input_seq.append(iseq)

# ネットワークの定義
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝播
output_data = []     # 出力データを格納するリスト
for iseq in input_seq:
    iseq = iseq.unsqueeze(0)    # 先頭にバッチサイズの次元1を加える
    oseq, _ = net(iseq)
    # スライスでシーケンスの最終状態のみ取り出し、squeezeで先頭のバッチサイズの次元1を除去してリストに追加
    output_data.append(oseq[:, -1, :].squeeze(0))

# 出力データの確認
for odata in output_data:
    print('output data:', odata.shape)

input sequence: torch.Size([8, 2])
input sequence: torch.Size([6, 2])
input sequence: torch.Size([10, 2])
output data: torch.Size([4])
output data: torch.Size([4])
output data: torch.Size([4])


## シーケンス長が揃っているデータにtorch.nn.LSTＭを使用

In [9]:
batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# 入力データのサンプル
input_seq = torch.randn(batch_size, seq_len, input_size)
print('input sequence:', input_seq.shape)

# ネットワークの定義
net = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)

# 順伝播
output_seq, _ = net(input_seq)
output_data = output_seq[:, -1, :] # スライスでシーケンスの最終状態のみ使う

# 出力データの確認
print('output data:', output_data.shape)

input sequence: torch.Size([20, 10, 2])
output data: torch.Size([20, 4])


## シーケンス長が揃っているデータにtorch.nn.LSTＭＣｅｌｌを使用

In [10]:
batch_size = 20
seq_len = 10

input_size = 2
hidden_size = 4

# 入力データのサンプル
input_seq = torch.randn(batch_size, seq_len, input_size)
print("input sequence：", input_seq.size())


# ネットワークの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block_a = nn.LSTMCell(input_size, hidden_size)  # LSTMブロックA
        self.block_b = nn.LSTMCell(hidden_size, hidden_size)  # LSTMブロックB

    def forward(self, x, hx_a0, cx_a0, hx_b0, cx_b0):
        '''
        :param x: 現時刻の入力シーケンスの状態
        :param hx_a0: 前時刻のLSTMブロックAの隠れ層の状態
        :param cx_a0: 前時刻のLSTMブロックAのセル状態
        :param hx_b0: 前時刻のLSTMブロックBの隠れ層の状態
        :param cx_b0: 前時刻のLSTMブロックBのセル状態
        :return: 現時刻のLSTMブロックAの隠れ層の状態とセル状態、現時刻のLSTMブロックBの隠れ層の状態とセル状態
        '''
        hx_a1, cx_a1 = self.block_a(x, (hx_a0, cx_a0))  # 現時刻のLSTMブロックAの隠れ層の状態とセル状態
        hx_b1, cx_b1 = self.block_b(hx_a1, (hx_b0, cx_b0))  # 現時刻のLSTMブロックBの隠れ層の状態とセル状態
        return hx_a1, cx_a1, hx_b1, cx_b1


net = Net()

# 隠れ層の初期化
hx_a = torch.randn(batch_size, hidden_size)
cx_a = torch.randn(batch_size, hidden_size)
hx_b = torch.randn(batch_size, hidden_size)
cx_b = torch.randn(batch_size, hidden_size)

# 出力データを生成
for i in range(seq_len):
    hx_a, cx_a, hx_b, hx_b = net(input_seq[:, i, :], hx_a, cx_a, hx_b, cx_b)
output_data = hx_b     # シーケンスの最終状態のみ取り出し出力とする

# 出力データの確認
print('output data:', output_data.shape)

input sequence： torch.Size([20, 10, 2])
output data: torch.Size([20, 4])
